In [7]:
import numpy as np
from draw_roulette import draw_roulette

# # Cover
# d = draw_roulette([11], (24, 3), handles=5, phase=30, h_phase=15, random_seed=0)
# # Américaine
# d = draw_roulette([0], americaine=True, handles=5, phase=0, h_phase=15, random_seed=3)
# Européenne
d = draw_roulette([0], americaine=False, handles=5, phase=0, h_phase=45, random_seed=3)
quality = 1200
d.setPixelScale(quality)  # Set number of pixels per geometry unit
d.rasterize()  # Display as PNG
# d.savePng('cover.png')
# d.savePng('americaine.png')
d.savePng('europeenne.png')
d
